#### Definition of Features
* SaleId: Satış ID
* SaleDate : Satış Tarihi
* Price: Satış için ödenen fiyat
* ConceptName:Otel konsept bilgisi
* SaleCityName: Otelin bulunduğu şehir bilgisi
* CheckInDate: Müşterininotelegirişitarihi
* CInDay:Müşterinin otele giriş günü
* SaleCheckInDayDiff: Check in ile giriş tarihi gün farkı
* Season:Otele giriş tarihindeki sezon bilgisi

In [1]:
import numpy as np
import pandas as pd
!pip install openpyxl
!update pandas

D:\anaconda\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


'update' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
pip install openpyxl --upgrade --pre

  Using cached https://files.pythonhosted.org/packages/e2/5e/1fe4ea74f5c0afc681cbb1f34836fa251280c5aa3012dc803f6aac1382d6/openpyxl-3.2.0b1-py2.py3-none-any.whl
  Found existing installation: openpyxl 3.0.0
    Uninstalling openpyxl-3.0.0:
      Successfully uninstalled openpyxl-3.0.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install openpyxl==3.0

  Found existing installation: openpyxl 3.2.0b1
    Uninstalling openpyxl-3.2.0b1:
      Successfully uninstalled openpyxl-3.2.0b1


In [49]:
df = pd.read_excel("miuul_gezinomi.xlsx")
df.head()

,SaleId,SaleDate,CheckInDate,Price,ConceptName,SaleCityName,CInDay,SaleCheckInDayDiff,Seasons
0,415122,2022-12-03,2022-12-03,79.304029,Herşey Dahil,Antalya,Saturday,0,Low
1,415103,2022-12-03,2022-12-03,45.970696,Yarım Pansiyon,Antalya,Saturday,0,Low
2,404034,2022-09-12,2022-09-13,77.838828,Herşey Dahil,Antalya,Tuesday,1,High
3,415094,2022-12-03,2022-12-10,222.710623,Yarım Pansiyon,İzmir,Saturday,7,Low
4,414951,2022-12-01,2022-12-03,140.476190,Yarım Pansiyon,İzmir,Saturday,2,Low


In [50]:
def check_df(dataframe, head=5):
    print("##### Shape #####")
    print(dataframe.shape)
    print("##### Types #####")
    print(dataframe.dtypes)
    print("##### NA #####")
    print(dataframe.isnull().sum())
check_df(df)

##### Shape #####
(59164, 9)
##### Types #####
SaleId                         int64
SaleDate              datetime64[ns]
CheckInDate           datetime64[ns]
Price                        float64
ConceptName                   object
SaleCityName                  object
CInDay                        object
SaleCheckInDayDiff             int64
Seasons                       object
dtype: object
##### NA #####
SaleId                 0
SaleDate               0
CheckInDate            0
Price                 13
ConceptName            0
SaleCityName           0
CInDay                 0
SaleCheckInDayDiff     0
Seasons                0
dtype: int64


In [51]:
df["SaleCityName"].nunique()

6

In [52]:
#Frequency of cities
df["SaleCityName"].value_counts() 

Antalya    31649
Muğla      10662
Aydın      10646
Diğer       3245
İzmir       2507
Girne        455
Name: SaleCityName, dtype: int64

In [53]:
df["ConceptName"].nunique()

3

In [54]:
#How many sales were achieved from which concept?
df["ConceptName"].value_counts()

Herşey Dahil      53186
Yarım Pansiyon     3559
Oda + Kahvaltı     2419
Name: ConceptName, dtype: int64

In [55]:
df.groupby("SaleCityName").agg({"Price" : "sum"} )

,Price
SaleCityName,
Antalya,2.041911e+06
Aydın,5.732960e+05
Diğer,1.545723e+05
Girne,2.706503e+04
Muğla,6.658422e+05
İzmir,1.659348e+05


In [56]:
df.groupby("ConceptName").agg({"Price" : "sum"} )

,Price
ConceptName,
Herşey Dahil,3.332911e+06
Oda + Kahvaltı,1.213084e+05
Yarım Pansiyon,1.744024e+05


In [57]:
df.groupby("SaleCityName").agg({"Price" : "mean"} )

,Price
SaleCityName,
Antalya,64.521474
Aydın,53.855896
Diğer,47.707497
Girne,59.483579
Muğla,62.461746
İzmir,66.267905


In [58]:
df.groupby("ConceptName").agg({"Price" : "mean"})

,Price
ConceptName,
Herşey Dahil,62.672260
Oda + Kahvaltı,50.252010
Yarım Pansiyon,49.030742


In [59]:
df.groupby(["SaleCityName", "ConceptName"]).agg({"Price" : "mean"})

Price
SaleCityName ConceptName              
Antalya      Herşey Dahil    64.519028
             Oda + Kahvaltı  63.504816
             Yarım Pansiyon  67.190613
Aydın        Herşey Dahil    53.995467
             Oda + Kahvaltı  34.458787
             Yarım Pansiyon  30.016543
Diğer        Herşey Dahil    84.771093
             Oda + Kahvaltı  37.599082
             Yarım Pansiyon  42.113440
Girne        Herşey Dahil    97.680970
             Oda + Kahvaltı  39.775536
             Yarım Pansiyon  53.247838
Muğla        Herşey Dahil    63.020053
             Oda + Kahvaltı  59.037763
             Yarım Pansiyon  45.120924
İzmir        Herşey Dahil    74.701519
             Oda + Kahvaltı  41.320342
             Yarım Pansiyon  59.610053

In [60]:
bins = [-1, 7, 30, 90, df['SaleCheckInDayDiff'].max()]
labels = ["Last Minuters", "Potential Planners", "Planners", "Early Bookers"]
df['EB_Score'] = pd.cut(df['SaleCheckInDayDiff'], bins=bins, labels=labels, include_lowest=True)
#df.head(50).to_excel("eb_score.xlsx", index=False)

In [61]:
df.head()

,SaleId,SaleDate,CheckInDate,Price,ConceptName,SaleCityName,CInDay,SaleCheckInDayDiff,Seasons,EB_Score
0,415122,2022-12-03,2022-12-03,79.304029,Herşey Dahil,Antalya,Saturday,0,Low,Last Minuters
1,415103,2022-12-03,2022-12-03,45.970696,Yarım Pansiyon,Antalya,Saturday,0,Low,Last Minuters
2,404034,2022-09-12,2022-09-13,77.838828,Herşey Dahil,Antalya,Tuesday,1,High,Last Minuters
3,415094,2022-12-03,2022-12-10,222.710623,Yarım Pansiyon,İzmir,Saturday,7,Low,Last Minuters
4,414951,2022-12-01,2022-12-03,140.476190,Yarım Pansiyon,İzmir,Saturday,2,Low,Last Minuters


In [62]:
df.groupby(["SaleCityName", "ConceptName", "EB_Score"]).agg({"Price" : ["mean","count"]})

Price       
                                                     mean  count
SaleCityName ConceptName    EB_Score                            
Antalya      Herşey Dahil   Last Minuters       62.752916  14148
                            Potential Planners  64.903700   8874
                            Planners            67.879960   4490
                            Early Bookers       66.494885   3281
             Oda + Kahvaltı Last Minuters       65.353740    503
...                                                   ...    ...
İzmir        Oda + Kahvaltı Early Bookers       66.928111      3
             Yarım Pansiyon Last Minuters       57.788126    636
                            Potential Planners  64.804043    198
                            Planners            60.507887     47
                            Early Bookers       65.905583     14

[72 rows x 2 columns]

In [63]:
df.groupby(["SaleCityName", "ConceptName", "Seasons"]).agg({"Price" : ["mean","count"]})

Price       
                                           mean  count
SaleCityName ConceptName    Seasons                   
Antalya      Herşey Dahil   High      64.920065  27126
                            Low       61.552427   3667
             Oda + Kahvaltı High      66.267969    303
                            Low       60.666730    295
             Yarım Pansiyon High      73.259029    118
                            Low       62.001678    138
Aydın        Herşey Dahil   High      54.946560  10103
                            Low       33.680699    473
             Oda + Kahvaltı High      30.387530     27
                            Low       44.451874     11
             Yarım Pansiyon High      32.626152     20
                            Low       25.271799     11
Diğer        Herşey Dahil   High      83.787273    364
                            Low       87.310882    141
             Oda + Kahvaltı High      34.839543    349
                            Low       39.959569    408
             Yarım Pansiyon High      39.731916    516
                            Low       42.953978   1462
Girne        Herşey Dahil   High     103.935448     55
                            Low       90.935944     51
             Oda + Kahvaltı High      39.479630     66
                            Low       40.043067     73
             Yarım Pansiyon High      58.383668    100
                            Low       48.578903    110
Muğla        Herşey Dahil   High      63.384036   9582
                            Low       48.304086    237
             Oda + Kahvaltı High      61.264691    592
                            Low       37.774192     62
             Yarım Pansiyon High      45.823698    177
                            Low       32.681812     10
İzmir        Herşey Dahil   High      74.748363   1234
                            Low       74.308287    147
             Oda + Kahvaltı High      54.141797     86
                            Low       33.555235    142
             Yarım Pansiyon High      87.657303    118
                            Low       55.350626    777

In [64]:
df.groupby(["SaleCityName", "ConceptName", "CInDay"]).agg({"Price" : ["mean","count"]})

Price      
                                            mean count
SaleCityName ConceptName    CInDay                    
Antalya      Herşey Dahil   Friday     62.658940  4136
                            Monday     63.259143  6831
                            Saturday   64.415526  4741
                            Sunday     65.849177  3818
                            Thursday   62.892734  3898
...                                          ...   ...
İzmir        Yarım Pansiyon Saturday   52.504725   315
                            Sunday     98.934697    68
                            Thursday   52.484894    91
                            Tuesday    56.141381    73
                            Wednesday  54.064392    86

[126 rows x 2 columns]

In [78]:
agg_df = df.groupby(["SaleCityName", "ConceptName", "Seasons"]).agg({"Price":"mean"}).sort_values("Price", ascending=False)
agg_df.head(20)

Price
SaleCityName ConceptName    Seasons            
Girne        Herşey Dahil   High     103.935448
                            Low       90.935944
İzmir        Yarım Pansiyon High      87.657303
Diğer        Herşey Dahil   Low       87.310882
                            High      83.787273
İzmir        Herşey Dahil   High      74.748363
                            Low       74.308287
Antalya      Yarım Pansiyon High      73.259029
             Oda + Kahvaltı High      66.267969
             Herşey Dahil   High      64.920065
Muğla        Herşey Dahil   High      63.384036
Antalya      Yarım Pansiyon Low       62.001678
             Herşey Dahil   Low       61.552427
Muğla        Oda + Kahvaltı High      61.264691
Antalya      Oda + Kahvaltı Low       60.666730
Girne        Yarım Pansiyon High      58.383668
İzmir        Yarım Pansiyon Low       55.350626
Aydın        Herşey Dahil   High      54.946560
İzmir        Oda + Kahvaltı High      54.141797
Girne        Yarım Pansiyon Low       48.578903

In [79]:
agg_df.reset_index(inplace=True)
agg_df.head()

,SaleCityName,ConceptName,Seasons,Price
0,Girne,Herşey Dahil,High,103.935448
1,Girne,Herşey Dahil,Low,90.935944
2,İzmir,Yarım Pansiyon,High,87.657303
3,Diğer,Herşey Dahil,Low,87.310882
4,Diğer,Herşey Dahil,High,83.787273


In [83]:
#agg_df["sales_level_based"] = df["SaleCityName"]+ "_" + df["ConceptName"]+ "_" + df["Seasons"]
agg_df["sales_level_based"]= agg_df[["SaleCityName", "ConceptName", "Seasons"]].agg(lambda x: "_".join(x).upper(), axis=1)
agg_df.head()

,SaleCityName,ConceptName,Seasons,Price,sales_level_based
0,Girne,Herşey Dahil,High,103.935448,GIRNE_HERŞEY DAHIL_HIGH
1,Girne,Herşey Dahil,Low,90.935944,GIRNE_HERŞEY DAHIL_LOW
2,İzmir,Yarım Pansiyon,High,87.657303,İZMIR_YARIM PANSIYON_HIGH
3,Diğer,Herşey Dahil,Low,87.310882,DIĞER_HERŞEY DAHIL_LOW
4,Diğer,Herşey Dahil,High,83.787273,DIĞER_HERŞEY DAHIL_HIGH


In [86]:
agg_df["SEGMENT"] = pd.qcut(agg_df["Price"], 4, labels=["D", "C", "B", "A"])
#qcut kendisi price'ı 4'e bölüyor
agg_df.head(20)

,SaleCityName,ConceptName,Seasons,Price,sales_level_based,SEGMENT
0,Girne,Herşey Dahil,High,103.935448,GIRNE_HERŞEY DAHIL_HIGH,A
1,Girne,Herşey Dahil,Low,90.935944,GIRNE_HERŞEY DAHIL_LOW,A
2,İzmir,Yarım Pansiyon,High,87.657303,İZMIR_YARIM PANSIYON_HIGH,A
3,Diğer,Herşey Dahil,Low,87.310882,DIĞER_HERŞEY DAHIL_LOW,A
4,Diğer,Herşey Dahil,High,83.787273,DIĞER_HERŞEY DAHIL_HIGH,A
5,İzmir,Herşey Dahil,High,74.748363,İZMIR_HERŞEY DAHIL_HIGH,A
6,İzmir,Herşey Dahil,Low,74.308287,İZMIR_HERŞEY DAHIL_LOW,A
7,Antalya,Yarım Pansiyon,High,73.259029,ANTALYA_YARIM PANSIYON_HIGH,A
8,Antalya,Oda + Kahvaltı,High,66.267969,ANTALYA_ODA + KAHVALTI_HIGH,A
9,Antalya,Herşey Dahil,High,64.920065,ANTALYA_HERŞEY DAHIL_HIGH,B


In [87]:
agg_df.groupby("SEGMENT").agg({"Price": ["mean", "max", "sum"]})

Price                        
              mean         max         sum
SEGMENT                                   
D        33.366288   39.479630  300.296592
C        44.887654   54.141797  403.988887
B        60.274498   64.920065  542.470478
A        82.467833  103.935448  742.210498

In [88]:
agg_df.sort_values(by="Price")

,SaleCityName,ConceptName,Seasons,Price,sales_level_based,SEGMENT
35,Aydın,Yarım Pansiyon,Low,25.271799,AYDIN_YARIM PANSIYON_LOW,D
34,Aydın,Oda + Kahvaltı,High,30.387530,AYDIN_ODA + KAHVALTI_HIGH,D
33,Aydın,Yarım Pansiyon,High,32.626152,AYDIN_YARIM PANSIYON_HIGH,D
32,Muğla,Yarım Pansiyon,Low,32.681812,MUĞLA_YARIM PANSIYON_LOW,D
31,İzmir,Oda + Kahvaltı,Low,33.555235,İZMIR_ODA + KAHVALTI_LOW,D
30,Aydın,Herşey Dahil,Low,33.680699,AYDIN_HERŞEY DAHIL_LOW,D
29,Diğer,Oda + Kahvaltı,High,34.839543,DIĞER_ODA + KAHVALTI_HIGH,D
28,Muğla,Oda + Kahvaltı,Low,37.774192,MUĞLA_ODA + KAHVALTI_LOW,D
27,Girne,Oda + Kahvaltı,High,39.479630,GIRNE_ODA + KAHVALTI_HIGH,D
26,Diğer,Yarım Pansiyon,High,39.731916,DIĞER_YARIM PANSIYON_HIGH,C


In [89]:
new_user = "ANTALYA_YARIM PANSIYON_HIGH"
agg_df[agg_df["sales_level_based"] == new_user]

,SaleCityName,ConceptName,Seasons,Price,sales_level_based,SEGMENT
7,Antalya,Yarım Pansiyon,High,73.259029,ANTALYA_YARIM PANSIYON_HIGH,A
